In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
import shutil 
import warnings
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import argparse
import torchvision
from torch.autograd import Variable
from tqdm import tqdm
import array
import matplotlib.pyplot as plt
from PIL import Image
# Ignore warnings
warnings.filterwarnings("ignore")
plt.ion()   # interactive mode

In [2]:
#Rename the folder using this peice of code from case_number to a sequential number 

# path = '/home/harsh/DDSM/normal'
# files = os.listdir(path)
# i = 1

# for file in files:
#     os.rename(os.path.join(path, file), os.path.join(path, str(i)))
#     i = i+1
# path = '/home/harsh/DDSM/cancers'
# files = os.listdir(path)
# i = 1

# for file in files:
#     os.rename(os.path.join(path, file), os.path.join(path, str(i)))
#     i = i+1
# path = '/home/harsh/project/DDSM/benigns'
# files = os.listdir(path)
# i = 1

# for file in files:
#     os.rename(os.path.join(path, file), os.path.join(path, str(i)))
#     i = i+1
 


In [ ]:
path_to_ddsm = "/home/harsh/project/DDSM/"

for root, subFolders, file_names in os.walk(path_to_ddsm):
    for file_name in file_names:
        if ".LJPEG" in file_name:
            ljpeg_path = os.path.join(root, file_name)
            out_path = os.path.join(root, file_name)
            out_path = out_path.split('.LJPEG')[0] + ".jpg"
            
            cmd = './ljpeg.py "{0}" "{1}" --visual --scale 1.0'.format(ljpeg_path, out_path)
            os.system(cmd)
#             delete_file_with_ext(out_path, '.LJPEG', True)

print('done')

In [3]:
#It is used to delete file with different different extension present in the directory once you convert the LJPEG 
#jpg format 
def delete_file_with_ext(cur_dir, old_ext, sub_dirs=False):
    if sub_dirs:
        for root, dirs, files in os.walk(cur_dir):
            for filename in files:
                file_ext = os.path.splitext(filename)[1]
                if old_ext == file_ext:
                    oldname = os.path.join(root, filename)
                    os.remove(oldname)
    else:
        files = os.listdir(cur_dir)
        for filename in files:
            file_ext = os.path.splitext(filename)[1]
            if old_ext == file_ext:
                os.rename(filename, newfile)

In [35]:
#It will delete files in all the folder of train and val
delete_file_with_ext('/home/harsh/project/DDSM', '.OVERLAY', True)
delete_file_with_ext('/home/harsh/project/DDSM', '.ics', True)
delete_file_with_ext('/home/harsh/project/DDSM', '.LJPEG', True)
delete_file_with_ext('/home/harsh/project/DDSM', '.16_PGM', True)

In [4]:
model_conv = models.vgg16_bn(pretrained=False)

In [5]:
print (model_conv)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [6]:
# This function cut the images from the seqential folder of benigns, cancers and normal and paste it to the 
#parent directory i.e. benign, cancers and normal and then delete the sequential folder and then rename the images
#given in sequence, also it flips the LEFT image, so basically all are right after implementing this function
def check_word_type(filename):
    words = "LEFT"
    if words in filename: 
        return True
    
def flip_image(image_path, saved_location):
    """
    image_path: The path to the image to edit
    saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    rotated_image = image_obj.transpose(Image.FLIP_LEFT_RIGHT)
    rotated_image.save(saved_location)

def flip_cut_paste(paste_dir):
    for root, dirs, files in os.walk(paste_dir):
        for directory in dirs:
#             print(directory)
            update_dir = os.path.join(paste_dir,directory)
#             print(update_dir)
            files = os.listdir(update_dir)
            for filename in files:
                print(os.path.basename(filename))
                a = check_word_type(os.path.basename(filename))
                if a: 
                    image = (os.path.join(update_dir,os.path.basename(filename)))
                    img = flip_image(image, image);  
                shutil.move(os.path.join(update_dir,os.path.basename(filename)), paste_dir)
            shutil.rmtree(update_dir)
    files = os.listdir(paste_dir)
    i = 1
    for file in files:
        os.rename(os.path.join(paste_dir, file), os.path.join(paste_dir, str(i)+'.jpg'))
        i = i+1


In [10]:
flip_cut_paste('/home/harsh/project/DDSM/train/benigns')
flip_cut_paste('/home/harsh/project/DDSM/train/cancers')
flip_cut_paste('/home/harsh/project/DDSM/train/normal')
flip_cut_paste('/home/harsh/project/DDSM/val/benigns')
flip_cut_paste('/home/harsh/project/DDSM/val/cancers')
flip_cut_paste('/home/harsh/project/DDSM/val/normal')

In [7]:
# to change the out feature of the model to 3
# num_ftrs = model_conv.fc.in_features   #for resnet18
num_ftrs = model_conv.classifier._modules['6'].in_features    #for VGG16_BN
print (num_ftrs)
model_conv.classifier._modules['6'] = nn.Linear(num_ftrs, 3)    #for VGG16_BN
# model_conv.fc = nn.Linear(num_ftrs, 3)     #for resnet18
# print (model_conv.fc.out_features)       #for resnet18   
print (model_conv.classifier._modules['6'].out_features)        #for VGG16_BN

4096
3


In [8]:
data_dir = "/home/harsh/project/DDSM/"
input_shape = 224
batch_size = 64
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = (224,224)
use_parallel = True
use_gpu = True
epochs = 50
data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=90),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
        transforms.Resize(scale),
        transforms.CenterCrop(input_shape),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),}

print(data_transforms)

{'val': Compose(
    Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'train': Compose(
    Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    RandomRotation(degrees=(-90, 90), resample=False, expand=False)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)}


In [9]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    print("MIXUP".format(mixup))
    since = time.time()
    global l
    global e 
    global a 
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                # get the inputs
                inputs, labels = data

                #augementation using mixup
                if phase == 'train' and mixup:
                    inputs = mixup_batch(inputs, alpha)
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)
            #print("the running loss is")
            #print(running_loss)
            #print("the running corrects is")
            #print(running_corrects)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = float(running_corrects) / float(dataset_sizes[phase])
            #print(epoch_acc)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            lo = '{:4f}'.format(epoch_loss)
            ac = '{:4f}'.format(epoch_acc)
            if phase == 'train':
                #print("the value of a is")
                #print(a)
                l.append(float(lo))
            if phase == 'val':
                a.append(float(ac))
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                
                
        print()

    time_elapsed = time.time() - since
    i = 0
    for i in range(num_epochs):
        e.append(float(i))
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [10]:
image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x),
                                      data_transforms[x]) for x in ['train', 'val']}
# print (image_datasets)
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=3) for x in ['train', 'val']}
print("Hello")
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print("Using CrossEntropyLoss")
criterion = nn.CrossEntropyLoss()

print("Using small learning rate with momentum")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)

print("Creating Learning rate scheduler")
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

print("Training the model begun")
model_conv = model_conv.cuda()
l = array.array('f',[])
e = array.array('f',[])
a = array.array('f',[])
model_ft = train_model(model_conv, dataloaders, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=epochs)
plt.plot(e,l)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(a,l)
plt.xlabel('Epoch')
plt.ylabel('accuracy')

plt.show()

  0%|          | 0/6 [00:00<?, ?it/s]

Hello
Using CrossEntropyLoss
Using small learning rate with momentum
Creating Learning rate scheduler
Training the model begun
MIXUP
Epoch 0/49
----------


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [31]:
torch.save(model_conv, 'model.pt')
